# Ноутбук для проекта

https://github.com/MaryIzo/taro_bot/tree/master   

https://colab.research.google.com/drive/1fbu1NBi_hrWDR0kJu32jvdVpFd8tugXA?usp=sharing   

Описание проекта:   
Часто бывает такое, что нужно принять какое-то важдое решение и "посоветоваться" с судьбой. Основываясь вероятностях и случайности и возможности человека обратиться с каким-то вопросом к искуственному интеллекту.  
Последнее время в интернете стали очень популярны гадания на картах таро, которые окружают себя ореолом таинственности, но по сути основаны на случайном выпадении карт в ответ на ваши вопросы.   
Давайте попробуем повторить это.  
    
План выполнения:   
1.  Добавим в модель промт, который превращает ее в гадалку
2.  Разобьем все возможные гадания на несколько вариантов: карта на день, прошлое настоящее будущее, вопрос, кельтский крест. От типа расклада зависит количество карт.    
3.  Используем RAG и случайность для того чтобы возвращать ответ на вопрос.  
4.  Надо разбить текст на части в которых есть:
     - описание каждого типа расклада
     - описание всех возможных карт
     - картинки всех возможных карт (можно попробовать использовать мультимодальную модель)


<center> <img src='https://github.com/a-milenkin/LLM_practical_course/blob/main/images/RAG.png?raw=1' width="800" height="250">


### Импорты всех необходимых библиотек

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install openai langchain tiktoken langchain-openai langchain-community sentence_transformers faiss-cpu pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 639.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 529.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 464.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 764.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 585.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 567.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 751.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40

In [ ]:
# !pip install chromadb

In [ ]:
!pip install transformers requests

In [ ]:
from typing import List
from pypdf import PdfReader
import pandas as pd
from tqdm import tqdm
import numpy as np
import requests
# import chromadb
import random
import pickle

In [ ]:
import pypdf

In [ ]:
!pip freeze | grep requests

requests==2.32.3
requests-oauthlib==1.3.1


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import FAISS
from google.colab import drive

## Предобработка текстов

### Собираем тексты для векторной базы

In [ ]:
drive.mount('/content/drive')

In [ ]:
# creating a pdf reader object
reader = PdfReader('drive/MyDrive/taro_bot/The Ultimate Guide to Tarot - A Beginner.pdf')
number_of_pages = len(reader.pages)

# printing number of pages in pdf file
print('Number of pages:', number_of_pages)

Number of pages: 493


In [ ]:
docs = []
for j in range(41):
  # getting a specific page from the pdf file
  page = reader.pages[j]
  # extracting text from page
  text = page.extract_text()
  docs.append(text.replace('\t', ' ').replace('\n', ' '))

In [ ]:
print(len(docs))

41


In [ ]:
str_docs = '\n'.join(docs)

In [ ]:
x = [len(one_a) for one_a in str_docs.split('\n')]
print('Средняя длина абзаца', np.mean(x))

Средняя длина абзаца 820.6341463414634


In [ ]:
print(len(str_docs))

33686


### Сформируем тексты и картинки для всех карт

In [ ]:
card_images = [
    45, 53, 62, 69, 78, 87, 96, 104, 113, 121, 129, 138, 147, 156, 164,
    173, 182, 191, 199, 208, 217, 226, 239, 243, 247, 251, 255, 259, 263, 266,
    267, 271, 275, 279, 283, 287, 291, 295, 299, 303, 307, 311, 315, 319, 323,
    327, 331, 335, 339, 343, 347, 351, 355, 359, 363, 367, 371,
    375, 379, 383, 387, 391, 395, 399, 403, 407, 411, 415, 419, 423,
    427, 431, 435, 439, 443, 447, 451, 455, 459
    ]
end_idx = 472

In [ ]:
def save_page_image(page, name):
  for count, image_file_object in enumerate(page.images):
    with open(f"""drive/MyDrive/taro_bot/card_images/{name}""" + "__" + f"""{str(count) + image_file_object.name}""", "wb") as fp:
        fp.write(image_file_object.data)
    return name + "__" + f"""{str(count) + image_file_object.name}"""

def get_image_name(text):
  return text.split('\n')[0].lower().replace(' ', '_').replace(',', '').replace('—', '')

def get_card_description(short_description, img_name_idx, next_img_idx):
    card_description = [short_description]
    for k in range(img_name_idx+1, next_img_idx-1):
      page = reader.pages[k]
      text = page.extract_text()
      card_description.append(text.replace('\t', ' '))
    return card_description

### Сохраняем все картинки в папочку card_images

In [ ]:
cards_dictionary = {}

for j in range(len(card_images)):
  img = card_images[j]
  img_idx = img - 1
  img_name_idx = img - 2
  if j < len(card_images)-1:
    next_img_idx = card_images[j+1] - 1
  else:
    next_img_idx = end_idx - 1

  page = reader.pages[img_name_idx]
  short_description = page.extract_text().replace('\t', ' ')
  name = get_image_name(short_description)
  if name == 'number_of_earthand_so_the_potential_to_bring_heaven_down_to_earth_to_manifest':
    name = 'number_of_earthand'

  card_description = get_card_description(short_description, img_name_idx, next_img_idx)
  final_name = save_page_image(reader.pages[img_idx], name)

  cards_dictionary[final_name] = card_description

In [ ]:
# print('\n'.join(card_description)

In [ ]:
cnt = 0
cards_meaning_splits = {}

for key, val in cards_dictionary.items():
  # print('Description len: ', len(val))

  card_description = ' '.join(val)
  if card_description.find('UPRIGHT MEANING') != -1:
    upight_meaning = card_description[
        card_description.find('UPRIGHT MEANING'):card_description.find('REVERSED MEANING')
        ]
  else:
    upight_meaning = card_description[
        card_description.find('UNDERSTANDING'):
        ]
  # print(upight_meaning)

  if (
      upight_meaning.find('Home') != -1
      and upight_meaning.find('Relationships') != -1
      and upight_meaning.find('Career and money') != -1
      ):
    meaning_split = [upight_meaning[:upight_meaning.find('Home')].replace('\n', ' '),
        upight_meaning[upight_meaning.find('Home'):upight_meaning.find('Relationships')].replace('\n', ' '),
        upight_meaning[upight_meaning.find('Relationships'):upight_meaning.find('Career and money')].replace('\n', ' '),
        upight_meaning[upight_meaning.find('Career and money'):].replace('\n', ' ')
    ]
  else:
    meaning_split = upight_meaning.replace('\n', ' ').split('.')

  cards_meaning_splits[key] = meaning_split

  cnt += 1
  # for split in meaning_split:
  #   print(cnt, split)
  # print()

In [ ]:
cards_splitted = list(cards_meaning_splits.values())

In [ ]:
with open('drive/MyDrive/taro_bot/cards_meaning_splits.pkl', 'wb') as f:
    pickle.dump(cards_meaning_splits, f)

### Создание коллекции для одной карты
Решила, что проще делать суммаризацию.

In [ ]:
# delete_mode = True
# cards_collections = {}
# chroma_client = chromadb.Client()

# for key, val in cards_dictionary.items():

#   if delete_mode:
#     chroma_client.delete_collection(name=key)
#   else:
#     collection = chroma_client.get_or_create_collection(name=key)

#     meaning_split = cards_meaning_splits[key]
#     collection.upsert(
#         documents=meaning_split,
#         ids=[f'id{i}' for i in range(len(meaning_split))]
#     )

In [ ]:
  # results = collection.query(
  #     query_texts=["What about relationtips"], # Chroma will embed this for you
  #     n_results=2  # how many results to return
  # )
  # print(results)

### Splitter из langchain

In [ ]:
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=250,
#     chunk_overlap=100,
#     length_function=len,
# )
# split_documents = splitter.create_documents([docs])
# len(split_documents)

In [ ]:
str_split_documents = docs.copy()
for sentence in cards_splitted:
  str_split_documents = str_split_documents + [sentence[0]]

In [ ]:
full_text = '\n'.join(str_split_documents)

In [ ]:
splitter = CharacterTextSplitter(
    separator="\n",  # символ-разделитель, по умолчанию переход к новому абзацу '\n\n'
    chunk_size=300,  # размер документа в символах
    chunk_overlap=10,  # насколько соседние документы могут перекрывать друг-друга
    length_function=len,  # функция, по которой считается размер документа
    is_separator_regex=False,  # является ли разделитель регулярным выражением
)
split_documents = splitter.create_documents([full_text])
str_split_documents = [split.page_content for split in split_documents]
len(split_documents)

85

In [ ]:
with open("drive/MyDrive/taro_bot/full_text.txt", "w") as fp:
  fp.write(full_text)

## Эмбеддинги от HuggingFace

In [ ]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/LaBSE", model_kwargs={"device": "cpu"}
)

<ipython-input-53-33cee1d2bef9>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf_embeddings_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set i

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
embeddings = hf_embeddings_model.embed_documents(
    str_split_documents
)

In [ ]:
with open('drive/MyDrive/taro_bot/hf_embeddings_model.pkl', 'wb') as f:
    pickle.dump(hf_embeddings_model, f)

## Сохраняем Vector Store

In [ ]:
db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)

db.save_local("drive/MyDrive/taro_bot/faiss_db")  # можно сохранить базу локально, указав путь

In [ ]:
# Load the model from the file using pickle
with open('drive/MyDrive/taro_bot/hf_embeddings_model.pkl', 'rb') as f:
    loaded_embeddings_model = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [ ]:
new_vector_store = FAISS.load_local(
    "drive/MyDrive/taro_bot/faiss_db", loaded_embeddings_model, allow_dangerous_deserialization=True
)

In [ ]:
# База готова, теперь можно делать к ней запросы
new_vector_store.similarity_search("What cards are arcana cards?")

[Document(metadata={}, page_content='The Minor Arcana Cards The remaining fifty-six cards are known collectively as the minor arcana, and they are arranged into four suits: Wands, Pentacles, Cups, and Swords. Each suit has fourteen cards, from Ace to Ten, plus four court cards: Page, Knight, Queen, and King. The four suits each have a ruling element, and each corresponds to specific areas of life:   Cups:  the element of Water; emotions and relationships'),
 Document(metadata={}, page_content='WHAT ABOUT REVERSALS? You will see that all the card interpretations in this book contain both upright and reversed card meanings. An upright card is just that—when you turn it over, it is upright. A card is known as reversed when it is upside down. There are lots of books dealing solely with reversed meanings. With a few exceptions, a reversed card’s meaning is generally more negative than the positive, or upright, meaning. However, many tarot professionals ignore reversals in a reading and just

In [ ]:
# База готова, теперь можно делать к ней запросы
db.similarity_search("What cards are arcana cards?")

[Document(page_content='UPRIGHT MEANING The upright meaning of the Ace of Wands is auspicious for every aspect of your life\nUPRIGHT MEANING The Two of Wands shows you making plans and moving forward, so travel arrangements are one meaning of the card'),
 Document(page_content='A Beginner’s Guide to the Cards, Spreads, and Revealing the Mystery of the Tarot LIZ DEAN author of  The Art of Tarot  and  The Golden Tarot'),
 Document(page_content='UPRIGHT MEANING The traditional meaning of the upright card is ruin\nUPRIGHT MEANING As an influence:  This card denotes useful information\nUPRIGHT MEANING As an influence:  Unpredictable, tempestuous times are ahead—you’re in for a roller coaster drama of highs and lows'),
 Document(page_content='UPRIGHT MEANING The Three of Wands is one of the tarot’s good-fortune cards\nUPRIGHT MEANING This lovely card reveals success after completion—a time for reward')]

## Определим Retriever

In [ ]:
retriever = db.as_retriever(
    search_type="mmr",  # тип поиска похожих документов
    k=1,  # количество возвращаемых документов (Default: 4)
    score_threshold=0.9,  # минимальный порог для поиска "similarity_score_threshold"
)

In [ ]:
retriver_answer = retriever.get_relevant_documents(
    "What cards are arcana cards?"
)

In [ ]:
retriver_str_answer = [split.page_content for split in retriver_answer]

In [ ]:
retriver_str_answer[0]

'The Minor Arcana Cards The remaining fifty-six cards are known collectively as the minor arcana, and they are arranged into four suits: Wands, Pentacles, Cups, and Swords. Each suit has fourteen cards, from Ace to Ten, plus four court cards: Page, Knight, Queen, and King. The four suits each have a ruling element, and each corresponds to specific areas of life:   Cups:  the element of Water; emotions and relationships'

In [ ]:
retriver_str_answer[1]

'A Beginner’s Guide to the Cards, Spreads, and Revealing the Mystery of the Tarot LIZ DEAN author of  The Art of Tarot  and  The Golden Tarot'

# Определим модель llm
https://huggingface.co/google/gemma-2-27b-it  - лучше всех отвечает, но ломается почему-то      
https://huggingface.co/microsoft/Phi-3-mini-4k-instruct   

https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3  - вторая по лучшести, все время выдаёт ответ  
https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407   
   
странная ошибка загрузки   
https://huggingface.co/google/gemma-2-27b-it    
https://huggingface.co/microsoft/Phi-3-mini-128k-instruct  

In [ ]:
import requests

# API_URL = "https://api-inference.huggingface.co/models/gpt2"
# API_URL = "https://api-inference.huggingface.co/models/google/gemma-2-27b-it"
# API_URL = "https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct"

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"


def query(payload, api_url=API_URL):
    headers = {"Authorization": f"Bearer ................"}
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()

output = query(
    {
    "inputs": "What the answer to the universe is",
})

print(output[0]['generated_text'])

What the answer to the universe is is one of those enduring questions that people have speculated on for centuries. The reason for the universe's existence is a deep philosophical question that has been addressed by religion, philosophy, and science. That being said, there isn't a definitive answer to this question because it is dependent upon one's own beliefs and interpretations.

In scientific terms, the reason for the universe's existence is that it is the result of a chain of events that began with the


In [ ]:
output = query({
    "inputs": """
    Find a peaceful, relaxed feeling. Feel comfortable and confident.
    Imagine that you can hear the universe and are a good fortune teller.
    Question {}
    """.format("What weather will be tomorrow?"),
})

print(output[0]['generated_text'])


    Find a peaceful, relaxed feeling. Feel comfortable and confident. 
    Imagine that you can hear the universe and are a good fortune teller.  
    Question What weather will be tomorrow?
    ---------------------------------------------
    Answer: I can hear your thoughts. You are asking the weather for tomorrow. The forecast for tomorrow is a bright and sunny day with a high of 70 degrees and a low of 40 degrees. Enjoy the sunshine!


In [ ]:
output = query({
    "inputs": """
    Imagine that you know everything about taro.
    Question: {}
    """.format("What cards are arcana cards?"),
})

print(output[0]['generated_text'])


    Imagine that you know everything about taro.
    Question: What cards are arcana cards?
    1. The Fool, The Magician, The High Priestess, The Empress, The Emperor, The Hierophant, The Lovers, The Chariot, Strength, The Hermit, The Wheel of Fortune, Justice, The Hanged Man, Death, Temperance, The Devil, The Tower, The Star, The Moon, The Sun, Judgement, and The World. These cards are part of the Major Arcana in a traditional


In [ ]:
output = query({
    "inputs": """
    Imagine that you know everything about taro. Return only the detailed answer.
    Use the following information if it is useful:
    {}
    Question: {}
    """.format(retriver_str_answer[0], "What cards are arcana cards?"),
})

print(output[0]['generated_text'])


    Imagine that you know everything about taro. Return only the detailed answer. 
    Use the following information if it is useful:
    The Minor Arcana Cards The remaining fifty-six cards are known collectively as the minor arcana, and they are arranged into four suits: Wands, Pentacles, Cups, and Swords. Each suit has fourteen cards, from Ace to Ten, plus four court cards: Page, Knight, Queen, and King. The four suits each have a ruling element, and each corresponds to specific areas of life:   Cups:  the element of Water; emotions and relationships
    Question: What cards are arcana cards?
     Answer: The Arcana cards are the remaining 56 cards in a standard tarot deck, including the four suits (Wands, Pentacles, Cups, and Swords) each having 14 numbered cards (Ace to Ten) and four court cards (Page, Knight, Queen, and King).


In [ ]:
output = query({
    "inputs": """
    Find a peaceful, relaxed feeling. Feel comfortable and confident.
    Imagine that you can hear the universe and are a good fortune teller.
    Summirize the following card:
    {}
    """.format(' '.join(random_card)),
})

print(output[0]['generated_text'])


    Find a peaceful, relaxed feeling. Feel comfortable and confident. 
    Imagine that you can hear the universe and are a good fortune teller. 
    Summirize the following card:
    UPRIGHT MEANING The traditional meaning of this Five of Pentacles is financial loss, so when it shows up in a reading, it can mean losing a job or a relationship or experiencing some other financial or emotional hardship  The positive aspect is that you find support from others in a similar position  These contacts may become good friends whom you may never have met in your usual circumstances  Consider new options, and you may discover another resource or approach that will help you see a way forward  However, this card often comes up in readings to show a fear of poverty and isolation, rather than actual poverty  It also commonly shows a fear of losing the security of home and/or the aftermath of a relationship breakup, with one partner feeling alone and depleted  It occasionally reveals the impact of 

# Основные функции

In [ ]:
def get_query(additional_prompt, main_input):
  """Function that returns response from hugging face API open model.
  """

  # Combine the additional prompt and main input with a special token
  query = f"{additional_prompt} [END_PROMPT] {main_input}"

  api_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

  headers = {
      "Authorization": "Bearer ................",
      "Content-Type": "application/json"
  }

  data = {
      "inputs": query,
      "parameters": {
          "max_length": 10000,  # Adjust as needed
          "temperature": 0.9  # Adjust as needed
      }
  }

  # Send the API request
  response = requests.post(api_url, headers=headers, json=data)

  # Parse the response
  if response.status_code == 200:
      output = response.json()
      # Extract the main output, excluding the additional prompt
      main_output = output[0]['generated_text'].split("[END_PROMPT]")[1].strip()
      main_output = main_output.replace(main_input, '').replace('\n', '').strip()
      main_output = main_output.split('.')[:3]

  else:
      print(f"Error: {response.status_code} - {response.text}")

  return '.'.join(main_output)

In [ ]:
def get_retriever(retriever_answer, question='What can you tell about taro?'):
  """
  Функция забирает суммаризацию на предсказание одной карты.
  Здесь лучше всего работает mistral.
  """


  additional_prompt = """
      Imagine that you know everything about taro. Return only the detailed answer.
      Use the following information if it is useful:
      {}
      """.format(' '.join(retriever_answer))
  main_input = "{}".format(question)

  return get_query(additional_prompt, main_input)

In [ ]:
def get_one_card_prediction(random_card, topic=''):
  """
  Функция забирает суммаризацию на предсказание одной карты.
  Здесь лучше всего работает mistral.
  """
  api_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

  additional_prompt = """
      Find a peaceful, relaxed feeling. Feel comfortable and confident.
      Imagine that you can hear the universe and are a good fortune teller.
      Make only three sentences.
      Try to use information:
      {}
      """.format(' '.join(random_card))
  main_input = "What is your Summirization? {}".format(topic)

  return get_query(additional_prompt, main_input)

In [ ]:
def get_yes_or_no(test_cards):
  """
  Функция  отвечает на вопрос Да или Нет.
  """
  api_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

  additional_prompt = """
      If you get three yeses, your answer is certain; two yes cards
      (with one no or neutral) mean a positive outcome is most likely, but it may take
      time to come about; and all no or a mix of no/neutral cards means the answer to
      your question is, of course, negative.

      Yes cards: All cards apart from those listed here as no, neutral, or exceptions.

      No cards
      Swords: Three, Five, Six, Seven, Eight, Nine, Ten, and Knight
      Cups: Five, Seven, and Eight
      Pentacles: Five
      Death
      The Devil
      The Tower
      The Moon

      Neutral cards
      Swords: Four
      Cups: Four
      The Hermit
      The Hanged Man

      Exceptions
      Two of Swords or Ten of Wands: The answer is not yet known.
      Five and Seven of Wands: The answer is yes, but you must fight for your
      prize.
      Your cards are:
      {}
      """.format(' '.join(test_cards))
  main_input = "What is your answer, yes or no? And why?"

  return get_query(additional_prompt, main_input)

# Основные формы гаданий

In [ ]:
the_taro_layout = "past, present, future"

if the_taro_layout == "question about taro":

  question = input()
  retriver_answer = retriever.get_relevant_documents(
    question
  )
  retriver_str_answer = [split.page_content for split in retriver_answer]
  answer = get_retriever(retriver_str_answer[0], question)

elif the_taro_layout == "one card":

  random_key = random.choice(list(cards_meaning_splits.keys()))
  additional_prompt = "One card for your events: "
  random_card = cards_meaning_splits[random_key]
  answer = ' '.join([additional_prompt, random_card])

elif the_taro_layout == "one card question":

  question = input()
  random_key = random.choice(list(cards_meaning_splits.keys()))
  random_card = cards_meaning_splits[random_key]
  answer = get_one_card_prediction(
      random_card,
      f' Answer on question {question}.'
      )

elif the_taro_layout == "past, present, future":

  times = {0: 'past', 1: 'present', 2: 'future'}
  answer = []
  for key, val in times.items():
    random_key = random.choice(list(cards_meaning_splits.keys()))
    random_card = cards_meaning_splits[random_key]
    card_answer = get_one_card_prediction(random_card, f' Answer in {val} tense.')
    answer.append( f"""{val} {card_answer}""")

  answer = ' '.join(answer)

elif the_taro_layout == "the celtic cross":

  questions = {
      0: "1 Your situation",
      1: "2 Responsibilities",
      2: "3 Limitations and the past",
      3: "4 What supports you",
      4: "5 What opposes you",
      5: "6 Achievements",
      6: "7 Attraction and relationships",
      7: "8 Work, health, and communication",
      8: "9 What is hidden",
      9: "10 The future environment; the outcome"
  }
  for key, val in questions.items():
    random_key = random.choice(list(cards_meaning_splits.keys()))
    random_card = cards_meaning_splits[random_key]

    answer = []
    card_answer = get_one_card_prediction(random_card, f' Answer on question {val}.')
    answer.append( f"""{val} {card_answer}""")

  ' '.join(answer)

elif the_taro_layout == "yes or no":

  test_cards = []
  for i in range(3):
    test_cards.append(random.choice(list(cards_meaning_splits.keys())))

  answer = get_yes_or_no(test_cards)

NameError: name 'random' is not defined

# Код бота
https://github.com/MaryIzo/taro_bot/tree/master